# Generate data using MOM6

### Setup dask

In [1]:
from dask_gateway import Gateway
gateway = Gateway()

# close existing clusters
open_clusters = gateway.list_clusters()
print(list(open_clusters))
if len(open_clusters)>0:
    for c in open_clusters:
        cluster = gateway.connect(c.name)
        cluster.shutdown()  

options = gateway.cluster_options()

options.worker_memory = 16 # 24 works fine for long term mean, but seasonal needs a bit more
# options.worker_cores = 8

options.environment = dict(
    DASK_DISTRIBUTED__SCHEDULER__WORKER_SATURATION="1.0"
)

# Create a cluster with those options
cluster = gateway.new_cluster(options)
client = cluster.get_client()
# cluster.adapt(5, 200)
cluster.scale(10)
client

[]


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/prod.aa26321b21c04990a7cc2a11f35bcfd3/status,


In [10]:
# If wanting to look at local server.
#import dask
#import distributed

#client = distributed.Client()
#client

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from xgcm import Grid
#from xmitgcm import open_mdsdataset
import xrft
#from scipy.ndimage import gaussian_filter
import gcm_filters as gcmf

import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
PERSISTENT_BUCKET = os.environ['PERSISTENT_BUCKET']

In [4]:
ds = xr.open_zarr(f'{PERSISTENT_BUCKET}/MOM6/Phillips2Layer/res4km_sponge10day/prog.zarr')

In [5]:
ds = ds.chunk({'Time':1,'xh':-1,'yh':-1,'xq':-1,'yq':-1})

In [42]:
Lfilter = np.array([400e3])
dx = 4e3 
coarsen_factor = 5 
coarsen_points= (Lfilter/coarsen_factor/dx).astype('int')

In [43]:
def filter_dataset(ds, Lfilter): 
    
    dx = ds.xh.diff('xh')[0].values*1e3
    wet_mask= (ds.h.isel(Time=0, zl=0)*0. + 1.).rename('wet_mask')
    
    filter_C = gcmf.Filter(filter_scale= Lfilter,  
                     dx_min = dx, 
                     filter_shape=gcmf.FilterShape.GAUSSIAN,
                     grid_type=gcmf.GridType.REGULAR_WITH_LAND,
                     grid_vars = {'wet_mask': wet_mask})
    
    xgrid = Grid(ds, coords={'X': {'center': 'xh', 'outer': 'xq'},
                         'Y': {'center': 'yh', 'outer': 'yq'},
                         'Z': {'center': 'zl', 'outer': 'zi'} },
             periodic=['X'])
    
    ds_filt = xr.Dataset() # For storing the filtered data 
    print('Started')
    ds_filt['h'] = filter_C.apply(ds.h, dims=['yh','xh']).rename('h')
    ds_filt['e'] = filter_C.apply(ds.e, dims=['yh','xh']).rename('e')
    u_c = xgrid.interp(ds.u.fillna(0), 'X')
    v_c = xgrid.interp(ds.v.fillna(0), 'Y')
    ds_filt['u'] = filter_C.apply(u_c, dims=['yh','xh']).rename('u')
    ds_filt['v'] = filter_C.apply(v_c, dims=['yh','xh']).rename('v')
    
    print('Filtered fields computed')
    
    ds_filt['uh'] = ds.h*u_c
    ds_filt['vh'] = ds.h*v_c
    
    ds_filt['uh_bar'] = filter_C.apply(ds_filt['uh'], dims=['yh','xh']).rename('uh_bar')
    ds_filt['vh_bar'] = filter_C.apply(ds_filt['vh'], dims=['yh','xh']).rename('vh_bar')
    
    ds_filt['ubar_hbar'] = (ds_filt.h*ds_filt.u)
    ds_filt['vbar_hbar'] = (ds_filt.h*ds_filt.v)
    
    ds_filt['uh_sg'] = ds_filt['uh_bar'] - ds_filt['ubar_hbar']
    ds_filt['vh_sg'] = ds_filt['vh_bar'] - ds_filt['vbar_hbar']

    print('Fluxes computed')
    
    ds_filt['dudx'] = xgrid.interp(xgrid.diff(ds_filt.u, 'X')/dx, 'X')
    ds_filt['dvdx'] = xgrid.interp(xgrid.diff(ds_filt.v, 'X')/dx, 'X')
    ds_filt['dudy'] = xgrid.interp(xgrid.diff(ds_filt.u, 'Y')/dx, 'Y')
    ds_filt['dvdy'] = xgrid.interp(xgrid.diff(ds_filt.v, 'Y')/dx, 'Y')
    ds_filt['slope_x'] = xgrid.interp(xgrid.diff(ds_filt.e, 'X')/dx, 'X')
    ds_filt['slope_y'] = xgrid.interp(xgrid.diff(ds_filt.e, 'Y')/dx, 'Y')
    
    print('Gradients computed')
    
    return ds_filt

In [44]:
def coarsen_dataset(ds, coarsen_points): 
    print('coarsening')
    return ds.coarsen(xh=coarsen_points, yh=coarsen_points).mean()

In [45]:
ds_filt_coarsen = coarsen_dataset(filter_dataset(ds, Lfilter[0]), coarsen_points[0]) 

Started
Filtered fields computed
Fluxes computed
Gradients computed
coarsening


In [46]:
%%time 
ds_filt_coarsen.load()

CPU times: user 8.5 s, sys: 343 ms, total: 8.84 s
Wall time: 3min 40s


<xarray.Dataset>
Dimensions:    (Time: 360, zl: 2, yh: 20, xh: 15, zi: 3)
Coordinates:
  * Time       (Time) object 0001-01-11 00:00:00 ... 0010-11-10 00:00:00
  * xh         (xh) float64 40.0 120.0 200.0 280.0 ... 1e+03 1.08e+03 1.16e+03
  * yh         (yh) float64 40.0 120.0 200.0 280.0 ... 1.4e+03 1.48e+03 1.56e+03
  * zl         (zl) float64 1.031e+03 1.033e+03
  * zi         (zi) float64 1.03e+03 1.032e+03 1.034e+03
Data variables: (12/18)
    h          (Time, zl, yh, xh) float64 580.1 580.1 ... 1.578e+03 1.577e+03
    e          (Time, zi, yh, xh) float32 0.2602 0.2605 0.2606 ... -2e+03 -2e+03
    u          (Time, zl, yh, xh) float32 -0.001071 -0.001075 ... -0.007009
    v          (Time, zl, yh, xh) float32 -0.002661 -0.002661 ... 0.0005938
    uh         (Time, zl, yh, xh) float64 -0.232 -0.2275 ... -20.17 -11.88
    vh         (Time, zl, yh, xh) float64 -1.018 -1.018 -1.054 ... 5.008 3.12
    ...         ...
    dudx       (Time, zl, yh, xh) float32 -1.049e-10 1.952e-11 ... 3.796e-09
    dvdx       (Time, zl, yh, xh) float32 -3.088e-11 4.466e-11 ... 1.357e-08
    dudy       (Time, zl, yh, xh) float32 7.988e-09 8.06e-09 ... -3.367e-08
    dvdy       (Time, zl, yh, xh) float32 -7.703e-09 -7.777e-09 ... -4.008e-09
    slope_x    (Time, zi, yh, xh) float32 3.174e-09 2.743e-09 ... 0.0 0.0
    slope_y    (Time, zi, yh, xh) float32 6.092e-06 6.092e-06 ... 0.0 0.0

In [47]:
fname = f'{PERSISTENT_BUCKET}/MOM6/Phillips2Layer/res4km_sponge10day/ml_data_'+str(int(Lfilter/1e3))+'km.zarr'
fname

'gs://leap-persistent/dhruvbalwada/MOM6/Phillips2Layer/res4km_sponge10day/ml_data_400km.zarr'

In [48]:
ds_filt_coarsen.to_zarr(fname)

In [49]:
test = xr.open_zarr(fname)

In [50]:
test

<xarray.Dataset>
Dimensions:    (Time: 360, zl: 2, yh: 20, xh: 15, zi: 3)
Coordinates:
  * Time       (Time) object 0001-01-11 00:00:00 ... 0010-11-10 00:00:00
  * xh         (xh) float64 40.0 120.0 200.0 280.0 ... 1e+03 1.08e+03 1.16e+03
  * yh         (yh) float64 40.0 120.0 200.0 280.0 ... 1.4e+03 1.48e+03 1.56e+03
  * zi         (zi) float64 1.03e+03 1.032e+03 1.034e+03
  * zl         (zl) float64 1.031e+03 1.033e+03
Data variables: (12/18)
    dudx       (Time, zl, yh, xh) float32 dask.array<chunksize=(180, 1, 20, 15), meta=np.ndarray>
    dudy       (Time, zl, yh, xh) float32 dask.array<chunksize=(180, 1, 20, 15), meta=np.ndarray>
    dvdx       (Time, zl, yh, xh) float32 dask.array<chunksize=(180, 1, 20, 15), meta=np.ndarray>
    dvdy       (Time, zl, yh, xh) float32 dask.array<chunksize=(180, 1, 20, 15), meta=np.ndarray>
    e          (Time, zi, yh, xh) float32 dask.array<chunksize=(180, 2, 10, 15), meta=np.ndarray>
    h          (Time, zl, yh, xh) float64 dask.array<chunksize=(180, 1, 20, 15), meta=np.ndarray>
    ...         ...
    uh_sg      (Time, zl, yh, xh) float64 dask.array<chunksize=(180, 1, 20, 15), meta=np.ndarray>
    v          (Time, zl, yh, xh) float32 dask.array<chunksize=(180, 1, 20, 15), meta=np.ndarray>
    vbar_hbar  (Time, zl, yh, xh) float64 dask.array<chunksize=(180, 1, 20, 15), meta=np.ndarray>
    vh         (Time, zl, yh, xh) float64 dask.array<chunksize=(180, 1, 20, 15), meta=np.ndarray>
    vh_bar     (Time, zl, yh, xh) float64 dask.array<chunksize=(180, 1, 20, 15), meta=np.ndarray>
    vh_sg      (Time, zl, yh, xh) float64 dask.array<chunksize=(180, 1, 20, 15), meta=np.ndarray>